### Kombinasi 9 :
* Delete Duplicate
* Delete Null
* Outlier capping winsorize sisi kanan
* Encoding 
* Standard scaler
* Feature selection -> K-Best

In [50]:
# import library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [51]:
df = pd.read_csv('../UFC_train.csv')

df.head()

,R_fighter,B_fighter,Referee,date,location,title_bout,weight_class,B_avg_KD,B_avg_opp_KD,B_avg_SIG_STR_pct,...,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age,Winner
0,Joe Riggs,Joe Doerksen,Steve Mazzagatti,2004-08-21,"Las Vegas, Nevada, USA",False,Middleweight,NaN,NaN,NaN,...,0,0,0,Southpaw,182.88,177.80,185.0,26.0,21.0,Red
1,Jorge Masvidal,Al Iaquinta,Keith Peterson,2015-04-04,"Fairfax, Virginia, USA",False,Lightweight,1.156250,0.000000,0.394141,...,0,1,0,Orthodox,180.34,187.96,170.0,27.0,30.0,Blue
2,Dan Stittgen,Stephen Thompson,Josh Rosenthal,2012-02-04,"Las Vegas, Nevada, USA",False,Welterweight,NaN,NaN,NaN,...,0,0,0,Orthodox,185.42,NaN,170.0,28.0,31.0,Blue
3,Josh Koscheck,Johny Hendricks,Kevin Mulhall,2012-05-05,"East Rutherford, New Jersey, USA",False,Welterweight,0.695312,0.000000,0.783359,...,6,3,0,Orthodox,177.80,185.42,170.0,28.0,34.0,Blue
4,John Dodson,Manvel Gamburyan,James Warring,2016-04-16,"Tampa, Florida, USA",False,Bantamweight,0.500000,0.266602,0.381462,...,3,0,1,Orthodox,160.02,167.64,135.0,34.0,31.0,Red


In [52]:
# Delete duplicate
print(f"Shape before dropping duplicates : {df.shape}") # Before dropping duplicates
df.drop_duplicates(inplace=True)
print(f"Shape after dropping duplicates: {df.shape}") # After dropping duplicates

Shape before dropping duplicates : (5410, 144)
Shape after dropping duplicates: (5410, 144)


In [53]:
def check_null(df):
    col_na = df.isnull().sum().sort_values(ascending=True)
    percent = col_na / len(df)
    missing_data = pd.concat([col_na, percent], axis=1, keys=['Total', 'Percent'])

    if (missing_data[missing_data['Total'] > 0].shape[0] == 0):
        print("Tidak ditemukan missing value pada dataset")
    else:
        print(missing_data[missing_data['Total'] > 0])

In [54]:
check_null(df)

                         Total   Percent
R_Weight_lbs                 2  0.000370
R_Height_cms                 4  0.000739
B_Weight_lbs                 8  0.001479
B_Height_cms                10  0.001848
R_Stance                    27  0.004991
...                        ...       ...
B_avg_GROUND_landed       1293  0.239002
B_avg_GROUND_att          1293  0.239002
B_avg_opp_CLINCH_landed   1293  0.239002
B_avg_TD_att              1293  0.239002
B_avg_HEAD_landed         1293  0.239002

[109 rows x 2 columns]


In [55]:
df.dropna(inplace=True)

In [56]:
check_null(df)

Tidak ditemukan missing value pada dataset


In [57]:
def check_outlier(df):
    Q1 = df.quantile(0.25)
    Q3 = df.quantile(0.75)

    # Menghitung RUB dan RLB.
    IQR = Q3 - Q1
    lower_limit = Q1 - 1.5*IQR
    upper_limit = Q3 + 1.5*IQR

    # Menampilkan banyaknya outlier pada atribut.
    outliers = (df < lower_limit) | (df > upper_limit)
    print ("Outlier pada tiap atribut:")
    print(outliers.sum())

    return outliers

In [58]:
# Split string and number
# Split dataframe into object type and number type
df_object = df.select_dtypes(include='object')
df_number = df.select_dtypes(include=np.number)

In [59]:
outliers = dict(check_outlier(df_number).sum())
print("\n\npercentage of outliers in each column:")
for key in outliers.keys():
    print(f"{key} = {outliers[key]/df_number[key].shape[0] * 100}%")

Outlier pada tiap atribut:
B_avg_KD                  67
B_avg_opp_KD             277
B_avg_SIG_STR_pct         91
B_avg_opp_SIG_STR_pct     82
B_avg_TD_pct               0
                        ... 
R_Height_cms              12
R_Reach_cms               55
R_Weight_lbs             172
B_age                     52
R_age                     12
Length: 134, dtype: int64


percentage of outliers in each column:
B_avg_KD = 1.9252873563218391%
B_avg_opp_KD = 7.959770114942528%
B_avg_SIG_STR_pct = 2.6149425287356323%
B_avg_opp_SIG_STR_pct = 2.3563218390804597%
B_avg_TD_pct = 0.0%
B_avg_opp_TD_pct = 2.442528735632184%
B_avg_SUB_ATT = 5.373563218390805%
B_avg_opp_SUB_ATT = 6.982758620689655%
B_avg_REV = 16.580459770114942%
B_avg_opp_REV = 15.057471264367816%
B_avg_SIG_STR_att = 2.1551724137931036%
B_avg_SIG_STR_landed = 2.21264367816092%
B_avg_opp_SIG_STR_att = 2.6436781609195403%
B_avg_opp_SIG_STR_landed = 2.7873563218390807%
B_avg_TOTAL_STR_att = 1.5517241379310345%
B_avg_TOTAL_STR_landed =

In [60]:
from scipy.stats.mstats import winsorize

# Identify columns with more than 0.5% outliers
outlier_columns = []
for key in outliers.keys():
    if outliers[key] / df_number[key].shape[0] > 0.005:
        outlier_columns.append(key)

# Winsorize the identified columns
for column in outlier_columns:
    df_number[column] = winsorize(df_number[column], limits=(0, 0.1))

# Combine df_object and df_number
df = pd.concat([df_object, df_number], axis=1)
df.head()

,R_fighter,B_fighter,Referee,date,location,weight_class,B_Stance,R_Stance,Winner,B_avg_KD,...,R_win_by_Decision_Split,R_win_by_Decision_Unanimous,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age
1,Jorge Masvidal,Al Iaquinta,Keith Peterson,2015-04-04,"Fairfax, Virginia, USA",Lightweight,Orthodox,Orthodox,Blue,0.785156,...,0,4,0,1,0,180.34,187.96,170.0,27.0,30.0
3,Josh Koscheck,Johny Hendricks,Kevin Mulhall,2012-05-05,"East Rutherford, New Jersey, USA",Welterweight,Southpaw,Orthodox,Blue,0.695312,...,1,4,5,3,0,177.80,185.42,170.0,28.0,34.0
4,John Dodson,Manvel Gamburyan,James Warring,2016-04-16,"Tampa, Florida, USA",Bantamweight,Orthodox,Orthodox,Red,0.500000,...,0,2,3,0,0,160.02,167.64,135.0,34.0,31.0
6,Tony Ferguson,Katsunori Kikuno,Yves Lavigne,2014-05-24,"Las Vegas, Nevada, USA",Lightweight,Orthodox,Orthodox,Red,0.000000,...,0,1,1,1,0,180.34,193.04,155.0,32.0,30.0
7,Richard Walsh,Viscardi Andrade,Marc Goddard,2016-03-19,"Brisbane, Queensland, Australia",Welterweight,Orthodox,Orthodox,Blue,0.500000,...,0,2,0,0,0,182.88,185.42,170.0,32.0,27.0


In [61]:
outliers = dict(check_outlier(df_number).sum())
print("\n\npercentage of outliers in each column:")
for key in outliers.keys():
    print(f"{key} = {outliers[key]/df_number[key].shape[0] * 100}%")

c:\Python312\Lib\site-packages\numpy\lib\function_base.py:4823: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  arr.partition(


c:\Python312\Lib\site-packages\numpy\lib\function_base.py:4823: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  arr.partition(


Outlier pada tiap atribut:
B_avg_KD                  0
B_avg_opp_KD              0
B_avg_SIG_STR_pct        39
B_avg_opp_SIG_STR_pct    26
B_avg_TD_pct              0
                         ..
R_Height_cms             12
R_Reach_cms              23
R_Weight_lbs              0
B_age                     0
R_age                    12
Length: 134, dtype: int64


percentage of outliers in each column:
B_avg_KD = 0.0%
B_avg_opp_KD = 0.0%
B_avg_SIG_STR_pct = 1.1206896551724137%
B_avg_opp_SIG_STR_pct = 0.7471264367816092%
B_avg_TD_pct = 0.0%
B_avg_opp_TD_pct = 0.0%
B_avg_SUB_ATT = 0.0%
B_avg_opp_SUB_ATT = 0.0%
B_avg_REV = 16.580459770114942%
B_avg_opp_REV = 15.057471264367816%
B_avg_SIG_STR_att = 0.0%
B_avg_SIG_STR_landed = 0.0%
B_avg_opp_SIG_STR_att = 0.0%
B_avg_opp_SIG_STR_landed = 0.0%
B_avg_TOTAL_STR_att = 0.0%
B_avg_TOTAL_STR_landed = 0.0%
B_avg_opp_TOTAL_STR_att = 0.0%
B_avg_opp_TOTAL_STR_landed = 0.0%
B_avg_TD_att = 0.0%
B_avg_TD_landed = 0.0%
B_avg_opp_TD_att = 0.0%
B_avg_opp_TD_land

In [62]:
df['gender'] = df['weight_class'].apply(lambda x: 'women' if 'women' in x.lower() else 'male')

In [63]:
df.head()

,R_fighter,B_fighter,Referee,date,location,weight_class,B_Stance,R_Stance,Winner,B_avg_KD,...,R_win_by_Decision_Unanimous,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age,gender
1,Jorge Masvidal,Al Iaquinta,Keith Peterson,2015-04-04,"Fairfax, Virginia, USA",Lightweight,Orthodox,Orthodox,Blue,0.785156,...,4,0,1,0,180.34,187.96,170.0,27.0,30.0,male
3,Josh Koscheck,Johny Hendricks,Kevin Mulhall,2012-05-05,"East Rutherford, New Jersey, USA",Welterweight,Southpaw,Orthodox,Blue,0.695312,...,4,5,3,0,177.80,185.42,170.0,28.0,34.0,male
4,John Dodson,Manvel Gamburyan,James Warring,2016-04-16,"Tampa, Florida, USA",Bantamweight,Orthodox,Orthodox,Red,0.500000,...,2,3,0,0,160.02,167.64,135.0,34.0,31.0,male
6,Tony Ferguson,Katsunori Kikuno,Yves Lavigne,2014-05-24,"Las Vegas, Nevada, USA",Lightweight,Orthodox,Orthodox,Red,0.000000,...,1,1,1,0,180.34,193.04,155.0,32.0,30.0,male
7,Richard Walsh,Viscardi Andrade,Marc Goddard,2016-03-19,"Brisbane, Queensland, Australia",Welterweight,Orthodox,Orthodox,Blue,0.500000,...,2,0,0,0,182.88,185.42,170.0,32.0,27.0,male


In [64]:
df['weight_class'] = df['weight_class'].str.replace('Women', '')

In [65]:
# Get all weight_class values
weight_class = df['weight_class'].unique()
weight_class

array(['Lightweight', 'Welterweight', 'Bantamweight', 'Flyweight',
       'LightHeavyweight', 'Strawweight', 'Featherweight', 'Middleweight',
       'Heavyweight', 'CatchWeight'], dtype=object)

In [66]:
# Create a dictionary to map weight_class values to numbers
weight_class_dict = {
    'CatchWeight' : 0,
    'Strawweight' : 1,
    'Flyweight' : 2,
    'Bantamweight' : 3,
    'Featherweight' : 4,
    'Lightweight' : 5,
    'Welterweight' : 6,
    'Middleweight' : 7,
    'LightHeavyweight' : 8,
    'Heavyweight' : 9,
    'OpenWeight' : 10,
}

gender_dict = {
    'male' : 1,
    'women' : 0
}

In [67]:
# Map each weight_class value to the correct number
df['weight_class'] = df['weight_class'].map(weight_class_dict)

In [68]:
# Map each gender value to the correct number
df['gender'] = df['gender'].map(gender_dict)

In [69]:
df.head()

,R_fighter,B_fighter,Referee,date,location,weight_class,B_Stance,R_Stance,Winner,B_avg_KD,...,R_win_by_Decision_Unanimous,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age,gender
1,Jorge Masvidal,Al Iaquinta,Keith Peterson,2015-04-04,"Fairfax, Virginia, USA",5,Orthodox,Orthodox,Blue,0.785156,...,4,0,1,0,180.34,187.96,170.0,27.0,30.0,1
3,Josh Koscheck,Johny Hendricks,Kevin Mulhall,2012-05-05,"East Rutherford, New Jersey, USA",6,Southpaw,Orthodox,Blue,0.695312,...,4,5,3,0,177.80,185.42,170.0,28.0,34.0,1
4,John Dodson,Manvel Gamburyan,James Warring,2016-04-16,"Tampa, Florida, USA",3,Orthodox,Orthodox,Red,0.500000,...,2,3,0,0,160.02,167.64,135.0,34.0,31.0,1
6,Tony Ferguson,Katsunori Kikuno,Yves Lavigne,2014-05-24,"Las Vegas, Nevada, USA",5,Orthodox,Orthodox,Red,0.000000,...,1,1,1,0,180.34,193.04,155.0,32.0,30.0,1
7,Richard Walsh,Viscardi Andrade,Marc Goddard,2016-03-19,"Brisbane, Queensland, Australia",6,Orthodox,Orthodox,Blue,0.500000,...,2,0,0,0,182.88,185.42,170.0,32.0,27.0,1


In [70]:
# Get the remaining object columns
df_object = df.select_dtypes(include='object')

# Perform one-hot encoding
df_encoded = pd.get_dummies(df_object)

# Combine the encoded columns with the original dataframe
df = pd.concat([df, df_encoded], axis=1)

# Drop the original object columns
df.drop(df_object.columns, axis=1, inplace=True)

df.head()

,weight_class,B_avg_KD,B_avg_opp_KD,B_avg_SIG_STR_pct,B_avg_opp_SIG_STR_pct,B_avg_TD_pct,B_avg_opp_TD_pct,B_avg_SUB_ATT,B_avg_opp_SUB_ATT,B_avg_REV,...,B_Stance_Orthodox,B_Stance_Southpaw,B_Stance_Switch,R_Stance_Open Stance,R_Stance_Orthodox,R_Stance_Southpaw,R_Stance_Switch,Winner_Blue,Winner_Draw,Winner_Red
1,5,0.785156,0.000000,0.394141,0.352422,0.239219,0.011484,0.156250,0.132812,0.000,...,True,False,False,False,True,False,False,True,False,False
3,6,0.695312,0.000000,0.600839,0.185547,0.088281,0.104375,0.093750,0.062500,0.000,...,False,True,False,False,True,False,False,True,False,False
4,3,0.500000,0.266602,0.381462,0.456558,0.429614,0.469570,1.351562,0.000244,0.125,...,True,False,False,False,True,False,False,False,False,True
6,5,0.000000,0.000000,0.470000,0.250000,0.750000,0.000000,0.000000,0.000000,0.000,...,True,False,False,False,True,False,False,False,False,True
7,6,0.500000,0.000000,0.460000,0.292500,0.125000,0.100000,0.000000,0.000000,0.000,...,True,False,False,False,True,False,False,True,False,False


In [71]:
df.to_csv('../Without Feature Selection/UFC_kombinasi9.csv', index=False)

In [72]:
from sklearn.feature_selection import SelectKBest, f_regression
# Perform feature selection using SelectKBest
selector = SelectKBest(k=20, score_func=f_regression)
X = df.drop('B_Reach_cms', axis=1) 
y = df['B_Reach_cms'] 
X_selected = selector.fit_transform(X, y)

# Get the selected feature names
selected_feature_names = X.columns[selector.get_support()]
print(selected_feature_names)
# Create a new dataframe with only the selected features
df_selected = pd.DataFrame(X_selected, columns=selected_feature_names)
df_selected.head()

Index(['weight_class', 'B_avg_SIG_STR_att', 'B_avg_opp_SIG_STR_att',
       'B_avg_TOTAL_STR_att', 'B_avg_opp_TOTAL_STR_att',
       'B_avg_opp_TOTAL_STR_landed', 'B_avg_opp_HEAD_att',
       'B_avg_opp_BODY_att', 'B_avg_opp_BODY_landed', 'B_avg_DISTANCE_att',
       'B_avg_opp_DISTANCE_att', 'B_avg_opp_DISTANCE_landed',
       'B_win_by_KO/TKO', 'B_Height_cms', 'B_Weight_lbs', 'R_win_by_KO/TKO',
       'R_Height_cms', 'R_Reach_cms', 'R_Weight_lbs', 'gender'],
      dtype='object')


,weight_class,B_avg_SIG_STR_att,B_avg_opp_SIG_STR_att,B_avg_TOTAL_STR_att,B_avg_opp_TOTAL_STR_att,B_avg_opp_TOTAL_STR_landed,B_avg_opp_HEAD_att,B_avg_opp_BODY_att,B_avg_opp_BODY_landed,B_avg_DISTANCE_att,B_avg_opp_DISTANCE_att,B_avg_opp_DISTANCE_landed,B_win_by_KO/TKO,B_Height_cms,B_Weight_lbs,R_win_by_KO/TKO,R_Height_cms,R_Reach_cms,R_Weight_lbs,gender
0,5.0,137.460938,89.953125,141.210938,91.882812,33.078125,83.210938,3.960938,2.070312,127.031250,88.695312,30.882812,3.0,177.80,155.0,0.0,180.34,187.96,170.0,1.0
1,6.0,39.218750,37.226562,51.500000,52.046875,28.671875,28.773438,6.656250,4.453125,25.703125,24.234375,6.359375,4.0,175.26,185.0,5.0,177.80,185.42,170.0,1.0
2,3.0,74.522461,93.299805,102.170410,125.859863,71.359619,76.181396,10.418213,9.236328,56.636475,69.770264,26.547607,0.0,165.10,135.0,3.0,160.02,167.64,135.0,1.0
3,5.0,48.000000,63.000000,104.000000,71.000000,24.000000,45.000000,12.000000,3.000000,37.000000,59.000000,15.000000,0.0,172.72,145.0,1.0,180.34,193.04,155.0,1.0
4,6.0,113.250000,81.000000,126.500000,92.750000,39.250000,69.250000,7.500000,3.750000,90.000000,68.500000,23.500000,1.0,180.34,170.0,0.0,182.88,185.42,170.0,1.0


In [73]:
y = df['B_Reach_cms']
y = y.reset_index(drop=True)

In [74]:
df_selected = pd.concat([df_selected, y], axis=1)

In [75]:
df_selected.to_csv('../Punya Andi/UFC_kombinasi9.csv', index=False)